In [ ]:
import numpy as np

NUM_COINS = 1000
FLIP_NUM = 10
TRIALS = 100000

def flip_coins(num_coins, num_flips):
  #flip num_coins coins num_flips times: 0 = heads, 1 = tails
  return np.random.randint(0, 2, size=(num_coins, num_flips))

def get_three_coins():
  #flip 1000 coins, 10 times each
  flips = flip_coins(NUM_COINS, FLIP_NUM)
  #calculate number of heads for each coin
  #heads are 0 so count tails and subtract from total
  heads_count = FLIP_NUM - flips.sum(axis=1)
  #get first coin heads count
  c_1 = heads_count[0] / FLIP_NUM
  #random coin chosen
  c_rand = heads_count[np.random.randint(0, NUM_COINS)] / FLIP_NUM
  #coin with minimum frequency of heads
  c_min = heads_count.min() / FLIP_NUM

  return(c_1, c_rand, c_min)

def main():
  v_1_avg, v_rand_avg, v_min_avg = 0, 0, 0
  #calculate v_1, v_rand, and v_min averages of 100,000 trials
  for i in range(TRIALS):
    v_1, v_rand, v_min = get_three_coins()
    v_1_avg += v_1
    v_rand_avg += v_rand
    v_min_avg += v_min
  #calculate the averages
  v_1_avg /= TRIALS
  v_rand_avg /= TRIALS
  v_min_avg /= TRIALS

  print(f'v1 avg: {v_1_avg}')
  print(f'v_rand avg: {v_rand_avg}')
  print(f'v_min avg: {v_min_avg}')

if __name__ == "__main__":
    main()

v1 avg: 0.4991370000000013
v_rand avg: 0.4992970000000033
v_min avg: 0.037671999999976696


In [ ]:
import numpy as np
SPACE = [-1, 1]

def generate_target_function():
    #randomly pick two points to define target function
    p1 = np.random.uniform(*SPACE, 2)
    p2 = np.random.uniform(*SPACE, 2)

    #define slope and intercept for the line
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
    y_intercept = p1[1] - slope * p1[0]

    #return the target function
    return lambda x: slope * x + y_intercept

#generate and store the target function
TARGET = generate_target_function()

#function to generate and sort the training data
def prepare_data(N, target_function):
    X_n = []
    y_n = []
    #generate the training data points randomly
    for i in range(N):
        new_pt = np.random.uniform(*SPACE, 2)  #generate random points
        y = target_function(new_pt[0])

        #append the feature vector [1, x, y] (1 is for the bias term)
        X_n.append([1, new_pt[0], new_pt[1]])
        #find the sign of the random data points
        y_n.append(np.sign(new_pt[1] - y))
    X_n = np.array(X_n)
    y_n = np.array(y_n).reshape(-1, 1)
    return X_n, y_n


#function to find the weights using linear regression
def linear_regression(X_n, y_n):
  #find the psuedo inverse
  psuedo_inverse = np.dot(np.linalg.inv(np.dot(X_n.T, X_n)), X_n.T)
  #calculate the weights
  w = np.dot(psuedo_inverse, y_n)
  return w.T

#function to classify points using weights based off of prediction
def classify(X_n, w):
  return np.sign(np.dot(X_n, w.T))

#function to run everything 1000 times and calculate E_in using learned weights
def run(N, trials, target_function):
  E_in_tot = 0
  #repeat for 1000 trials
  for i in range(trials):
    #generate training data
    X_n, y_n = prepare_data(N, target_function)
    w = linear_regression(X_n, y_n) #compute weights
    y_predicted = classify(X_n, w)
    #find E_in for the N(N=100) training data points
    E_in = np.mean(y_predicted != y_n)
    E_in_tot += E_in
  #calculate average E_in
  avg_E_in = E_in_tot / trials
  return avg_E_in


def main():
  N = 100 #num of training points
  trials = 1000 #num of trials
  avg_E_in = run(N, trials, TARGET)
  print(f'Average in-sample error: {avg_E_in}')

if __name__ == "__main__":
    main()

Average in-sample error: 0.02848999999999999


In [ ]:
#function to create the out of sample data and calculate E_out on it
def out_of_sample(target_function, w, tests):
  #generate a new set of test data
  X_test, y_test = prepare_data(tests, target_function)
  y_predicted = classify(X_test, w) #classify the test data
  #calculate out-of-sample error
  E_out = np.mean(y_predicted != y_test)
  return E_out

#function to run the entire process and calculate both E_in and E_out
def new_run(N, tests, trials, target_function):
  E_out_tot, E_in_tot = 0, 0
  #repeat for specified number of trials
  for i in range(trials):
    #generate in-sample training data
    X_n, y_n = prepare_data(N, target_function)
    #compute the weights using linear regression on the training data
    w = linear_regression(X_n, y_n)
    y_predicted_in = classify(X_n, w)
    E_in = np.mean(y_predicted_in != y_n) #calculate E_in
    E_in_tot += E_in
    E_out = out_of_sample(target_function, w, tests) #calculate E_out
    E_out_tot += E_out

  #compute avg E_in and E_out
  avg_E_in = E_in_tot / trials
  avg_E_out = E_out_tot / trials

  return avg_E_in, avg_E_out


def main():
  N = 100 #num of training data points
  trials = 1000 #num of trials to run
  tests = 1000 #num of out of sample points
  avg_E_in, avg_E_out = new_run(N, tests, trials, TARGET)
  print(f"avg in sample error: {avg_E_in}")
  print(f"avg out of sample error: {avg_E_out}")

if __name__ == "__main__":
    main()

avg in sample error: 0.029269999999999994
avg out of sample error: 0.036466999999999944


In [ ]:
max_iterations = 10000 #max iteration limit to avoid infinite loop

#function to find misclassified points
def misclassified(a, b):
    return np.where(a != b)[0]

#perceptron learning algorithm
def PLA(X_n, y_n, w):
  count = 0
  #repeat till convergence or max_iterations reached
  while count < max_iterations:
    #predict classifications based on current weights
    predictions = classify(X_n, w)
    #find misclassified points
    missed = misclassified(y_n, predictions).tolist()
    if not missed: #break if no misclassifications
      break

    #randomly select a misclassified point
    rand_idx = np.random.choice(missed)
    w = w + y_n[rand_idx] * X_n[rand_idx] #update weight vec

    count += 1
  return w, count

#function to run PLA over multiple trials and calculate avg iterations
def run_PLA(N, trials, target_function):
  total_iterations = 0
  #run for specified number of trials
  for i in range(trials):
    #generate training data
    X_n, y_n = prepare_data(N, target_function)
    #get initial weights from linear regression
    w_initial = linear_regression(X_n, y_n)
    #apply PLA and calculate num of iterations
    w, iterations = PLA(X_n, y_n, w_initial)
    total_iterations += iterations
  #find average number of iterations across all trials
  avg_iterations = total_iterations / trials

  return avg_iterations


def main():
  N = 10 #num of training points
  trials = 1000 #num of trials
  #run PLA experiment
  avg_iterations = run_PLA(N, trials, TARGET)
  print(f'avg number of iterations: {avg_iterations}')


if __name__ == "__main__":
    main()


avg number of iterations: 3.32


In [ ]:
import numpy as np

#define target function
def target_func(x1, x2):
  return np.sign(x1**2 + x2**2 - 0.6)

#function to generate random data points and create noise
def generate_data(N):
  #generate random points
  X = np.random.uniform(-1, 1, (N, 2))
  y = target_func(X[:, 0], X[:, 1])
  #introduce 10% noise by flipping the sign of 10% of the labels
  noise_indices = np.random.choice(N, size=int(0.1 * N), replace=False)
  y[noise_indices] *= -1
  #add bias (1) to the feature vector
  X = np.column_stack((np.ones(N), X))

  return X, y

#function to perform linear regression and calculate weights
def lin_regression(X, y):
  #find the psuedo inverse
  psuedo_inverse = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
  #calculate the weights
  w = np.dot(psuedo_inverse, y)
  return w.T

#function to classify data based on learned weights
def classify(X_n, w):
  return np.sign(np.dot(X_n, w.T))

#function to run experiment and caculate avg in-sample error
def run_expt(N, trials):
  E_in_tot = 0

  #repeate experiment specified number of times
  for i in range(trials):
    X, y = generate_data(N) #generate training data
    w = lin_regression(X, y) #compute weights
    y_predicted = classify(X, w) #predict classifications
    #calculate in-sample error
    E_in = np.mean(y_predicted != y)
    E_in_tot += E_in
  #return avg in-sample error
  return E_in_tot / trials


def main():
  N = 1000 #number of data points to generate
  trials = 1000 #number of trials
  avg_E_in = run_expt(N, trials) #get avg E_in
  print(f'avg in-sample error: {avg_E_in}')

if __name__ == "__main__":
    main()

avg in-sample error: 0.5036839999999996


In [ ]:
#function to apply nonlinear transformation
def nonlinear_transformation(X):
  x1 = X[:, 1]
  x2 = X[:, 2]
  #create new feature matrix as specified in problem
  X_transformed = np.column_stack((np.ones(X.shape[0]), x1, x2, x1 * x2, x1**2, x2**2))

  return X_transformed

#function to compare learned hypothesis with predefined hypotheses
def compare_hypothesis(X_transformed, y, w):
  hypotheses = [
    np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5]), #[a] hypothesis
    np.array([-1, -0.05, 0.08, 0.13, 1.5, 15]), #[b] hypothesis
    np.array([-1, -0.05, 0.08, 0.13, 15, 1.5]), #[c] hypothesis
    np.array([-1, -1.5, 0.08, 0.13, 0.05, 0.05]), #[d] hypothesis
    np.array([-1, -0.05, 0.08, 1.5, 0.15, 0.15]) #[e] hypothesis
  ]
  #get predictions from learned model
  model_predictions = classify(X_transformed, w)
  agreement_probs = []
  #evaluate predictions of each hypothesis
  for hypothesis in hypotheses:
    #calculate predictions of specified hypothesis
    hypothesis_predictions = np.sign(np.dot(X_transformed, hypothesis.T))
    #calculate agreement between model's prediction and hypothesis prediction
    agreement = np.mean(model_predictions == hypothesis_predictions)
    agreement_probs.append(agreement)

  return agreement_probs

#function to run experiment on specified number of trials and data points
def run(N, trials):
  total_agree_prob = np.zeros(5)
  #loop through specified number of trials
  for i in range(trials):
    X, y = generate_data(N) #generate training data
    X_transformed = nonlinear_transformation(X) #apply nonlinear transform
    w = lin_regression(X_transformed, y) #compute weights w/ linear regression
    #compare with predefined hypotheses
    agreement_probs = compare_hypothesis(X_transformed, y, w)
    total_agree_prob += agreement_probs
  #return avg agreement probabilities for all hypotheses
  return total_agree_prob / trials

def main():
  N = 1000 #number of data points
  trials = 1000 #number of trials
  #find agreement probabilities for all hypotheses
  agreement_probs = run(N, trials)
  for i, prob in enumerate(agreement_probs):
    print(f'hypothesis {i+1}: {prob}')

if __name__ == "__main__":
    main()

hypothesis 1: 0.962229000000001
hypothesis 2: 0.6635789999999997
hypothesis 3: 0.6629049999999983
hypothesis 4: 0.6319689999999993
hypothesis 5: 0.5598759999999997


IGNORE THIS ONE

In [ ]:
hypothesis = np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5])

def estimate_E_out(tests, trials):
  E_out_tot = 0
  E_in_tot = 0

  for i in range(trials):
    X_train, y_train = generate_data(tests)

    X_train_transformed = nonlinear_transformation(X_train)

    w_transformed = lin_regression(X_train_transformed, y_train)

    y_predicted_train = classify(X_train_transformed, w_transformed)

    E_in = np.mean(y_predicted_train != y_train)
    E_in_tot += E_in

    X_test, y_test = generate_data(tests)

    X_test_transformed = nonlinear_transformation(X_test)

    y_predicted = classify(X_test_transformed, w_transformed)

    E_out = np.mean(y_predicted != y_test)
    E_out_tot += E_out

  avg_E_in = E_in_tot / trials
  avg_E_out = E_out_tot / trials
  return avg_E_in, avg_E_out

def main():
  tests = 1000
  trials = 1000
  avg_E_in, avg_E_out = estimate_E_out(tests, trials)
  print(f'avg in sample error: {avg_E_in}')
  print(f'avg out of sample error: {avg_E_out}')

if __name__ == "__main__":
  main()

avg in sample error: 0.12379699999999978
avg out of sample error: 0.12649000000000007


In [ ]:
#define hypothesis from problem 9
hypothesis = np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5])

#function to estimate E_out
def estimate_E_out(tests, trials):
  E_out_tot = 0
  #evaluate over specified number of trials
  for i in range(trials):
    #generate a new test dataset
    X_test, y_test = generate_data(tests)
    # apply the nonlinear transformation to test data
    X_test_transformed = nonlinear_transformation(X_test)
    #use the predefined hypothesis to classify test data
    y_predicted = classify(X_test_transformed, hypothesis)
    #calculate out-of-sample error
    E_out = np.mean(y_predicted != y_test)
    E_out_tot += E_out
  #calculate the average out-of-sample error over all trials
  avg_E_out = E_out_tot / trials
  return avg_E_out

def main():
  tests = 1000  #number of test points to generate
  trials = 1000  #number of trials to average error
  avg_E_out = estimate_E_out(tests, trials)  #estimate out-of-sample error
  print(f'average out of sample error: {avg_E_out}')

if __name__ == "__main__":
    main()

average out of sample error: 0.1427290000000002
